In [ ]:
from gensim import __version__
print(__version__)

In [ ]:
from word_sense_induction import minimize 
from chinese_whispers import chinese_whispers, aggregate_clusters
from networkx import Graph
from gensim.models import KeyedVectors
from time import time 
import networkx as nx
import matplotlib.pyplot as plt
from pandas import read_csv
from glob import glob 
from collections import Counter
import codecs
from traceback import format_exc
import gzip
import logging
import gensim
from os.path import exists


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

TOPN = 50
verbose = True

try:
    wv
except NameError:
    wv = None


def get_sorted_vocabulary(vectors_fpath):
    with gzip.open(vectors_fpath, "rb") as in_f:
        vocabulary = []
        for i, line in enumerate(in_f):
            if i == 0: continue
            vocabulary.append( str(line, "utf-8").split(" ")[0] )

            
def save_to_gensim_format(wv, output_fpath):
    tic = time()
    wv.save(output_fpath)
    print("Saved in {} sec.".format(time()-tic))
    

def load_globally(word_vectors_fpath):
    global wv
    
    if not wv:
        print("Loading word vectors from:", word_vectors_fpath)
        tic = time()
        if word_vectors_fpath.endswith(".vec.gz"):
            wv = KeyedVectors.load_word2vec_format(word_vectors_fpath, binary=False, unicode_errors="ignore")
        else:
            wv = gensim.models.KeyedVectors.load(word_vectors_fpath)            
        print("Loaded in {} sec.".format(time()-tic))
    else:
        print("Using loaded word vectors.")

    return wv


def get_nns(target, topn=TOPN):
    nns = wv.most_similar(positive=[target], negative=[], topn=topn)
    nns = [(word, score) for word, score in nns if minimize(word) != minimize(target)]
    return nns


def in_nns(nns, word):
    for w, s in nns:
        if minimize(word) == minimize(w):
            return True
        
    return False 


def get_pair(first, second):
    pair_lst = sorted([first, second])
    sorted_pair = (pair_lst[0], pair_lst[1])
    return sorted_pair         


def get_disc_pairs(ego, topn=TOPN):  
    pairs = set()
    nns = get_nns(ego, topn)
    
    for i in range(len(nns)):
        topi = nns[i][0]
        nns_topi = get_nns(topi, topn) 
        nns_untopi = wv.most_similar(positive=[ego], negative=[topi], topn=topn)
        untopi = nns_untopi[0][0]
        if in_nns(nns, untopi): pairs.add(get_pair(topi, untopi))

    return pairs


def get_nodes(pairs):
    nodes = Counter()
    for src, dst in pairs:
        nodes.update([src])
        nodes.update([dst])
        
    return nodes


def list2dict(lst):
    return {p[0]: p[1] for p in lst}


def wsi(ego, topn=TOPN):
    tic = time()
    ego_network = Graph(name=ego)

    pairs = get_disc_pairs(ego, topn)
    nodes = get_nodes(pairs)   
    
    ego_network.add_nodes_from( [(node, {'size': size}) for node, size in nodes.items()] )
    
    for r_node in ego_network:
        related_related_nodes = list2dict(get_nns(r_node))
        related_related_nodes_ego = sorted(
            [(related_related_nodes[rr_node], rr_node) for rr_node in related_related_nodes if rr_node in ego_network],
            reverse=True)[:topn]
        
        related_edges = []
        for w, rr_node in related_related_nodes_ego:
            if get_pair(r_node, rr_node) not in pairs:
                related_edges.append( (r_node, rr_node, {"weight": w}) )
            else:
                print("Skipping:", r_node, rr_node)
        ego_network.add_edges_from(related_edges)

    chinese_whispers(ego_network, weighting="top", iterations=20)
    if verbose: print("{}\t{:f} sec.".format(ego, time()-tic))

    return {"network": ego_network,  "nodes": nodes}


def draw_ego(G, show=False, save_fpath=""):
    colors = [1. / G.node[node]['label'] for node in G.nodes()]
    sizes = [300. * G.node[node]['size'] for node in G.nodes()]  

    plt.clf()
    fig = plt.gcf()
    fig.set_size_inches(20, 20)

    nx.draw_networkx(G, cmap=plt.get_cmap('gist_rainbow'),
                     node_color=colors,
                     font_color='black',
                     node_size=sizes)

    if show: plt.show()
    if save_fpath != "":
        plt.savefig(save_fpath)
        
    fig.clf()
        

def get_target_words():
    words = set(["Maus", "Schlange"])

    for pairs_fpath in glob("/home/panchenko/eval/de*dataset"):
        df = read_csv(pairs_fpath, sep=";", encoding="utf-8")
        for i, row in df.iterrows():
            words.add(row.word1)
            words.add(row.word2)

    words = sorted(words)
    return words


def get_cluster_lines(G, nodes):
    lines = []
    labels_clusters = sorted(aggregate_clusters(G).items(), key=lambda e: len(e[1]), reverse=True)
    for label, cluster in labels_clusters:
        scored_words = []
        for word in cluster:
            scored_words.append( (nodes[word], word) )
        keyword = sorted(scored_words, reverse=True)[0][1]
        
        lines.append("{}\t{}\t{}\t{}\n".format(G.name, label, keyword, ", ".join(cluster)))
        
    return lines 


visualize = False
show_plot = False
eval_vocabulary = False
wv_fpath = "model/cc.de.300.vec.gz"
wv_gensim_fpath = wv_fpath + ".gensim"

if eval_vocabulary:
    words = {w: None for w in get_target_words()}
else:
    words = get_sorted_vocabulary(wv_fpath)

if not exists(wv_gensim_fpath):
    load_globally(wv_fpath)
    save_to_gensim_format(vw, wv_gensim_fpath)
else:
    load_globally(wv_gensim_fpath)
    
for topn in [50, 100, 200]: 
    output_fpath = wv_fpath + ".top{}.inventory.tsv".format(topn)
    with codecs.open(output_fpath, "w", "utf-8") as out:
        out.write("word\tcid\tkeyword\tcluster\n")
        for word in words:
            try:
                words[word] = wsi(word, topn=topn)
                if visualize:
                    plt_fpath = output_fpath + ".{}.png".format(word)
                    draw_ego(words[word]["network"], show_plot, plt_fpath)
                lines = get_cluster_lines(words[word]["network"], words[word]["nodes"])
                for l in lines: out.write(l)
            except KeyboardInterrupt:
                break
            except:
                print("Error:", word)
                print(format_exc())
    print("Output:", output_fpath)
    

2018-04-11 13:00:07,419 : INFO : loading projection weights from model/cc.de.300.vec.gz


Loading word vectors from: model/cc.de.300.vec.gz


In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
        
vocabulary = get_sorted_vocabulary("model/cc.de.300.vec.gz")

In [ ]:
import lo

In [ ]:
s = "klasf.vw"
s.endswith(".vw")

In [ ]:
%load_ext autoreload
%autoreload 2